In [13]:
import copy
import json
import sqlite3
import pandas as pd
import datetime,time
import os
import numpy as np

In [162]:
DB = 'airfare_scraped_data.db'
def report_query():
    ## 기본 쿼리 작성 - 7월 1일 이후 스크랩한 데이터중
    ## GMP,CJU 7월 18일 이후 자료 전체 리스트
    param_ = {}
    param_['scrap_date'] = '20170701' ## 7월 이후
    param_['dpt'] = 'CJU' ## 출도착지가 없으면 에러
    param_['arr'] = 'GMP'
    sql = '''
        select scrap_date, airline, flt, dpt_date, dpt, arr, dpt_time, arr_time, fare, tax1, tax2, seat
        from airfare_scraped_data
        where scrap_date >= '{scrap_date}00'
        and dpt='{dpt}' and arr='{arr}'
        and dpt_date >= '20170718' and dpt_date <= '20170724'
        '''
        #and dpt_date <= strftime('%Y%m%d','now','+{days} day')
    query = sql.format(**param_)
    print("query : ",query)
    return query
## 그룹화한 ndarray에서 최소값 찾기
def agg_min(arr):
    narr = arr.values
    narr = narr[narr > 0]
    if len(narr) <= 0:
        return 0
    else:
        return sorted(narr)[0]
## 그룹화한 ndarray에서 평균값 찾기
## 동일한 가격들을 제거하여 유니크한 상태에서 평균값 계산
def agg_mean(arr):
    #print(arr)
    narr = np.unique(arr.values)
    narr = narr[narr > 0]
    if len(narr) <= 0:
        return 0
    else:
        return narr.mean()

In [4]:
report_query()

query :  
        select scrap_date, airline, flt, dpt_date, dpt, arr, dpt_time, arr_time, fare, tax1, tax2, seat
        from airfare_scraped_data
        where scrap_date >= '2017070100'
        and dpt='GMP' and arr='CJU'
        and dpt_date >= '20170718' and dpt_date <= '20170724'
        


"\n        select scrap_date, airline, flt, dpt_date, dpt, arr, dpt_time, arr_time, fare, tax1, tax2, seat\n        from airfare_scraped_data\n        where scrap_date >= '2017070100'\n        and dpt='GMP' and arr='CJU'\n        and dpt_date >= '20170718' and dpt_date <= '20170724'\n        "

In [163]:
query = report_query()
query = "select airline, dpt_date, dpt, arr, flt, dpt_time, fare from(" +\
    query +\
    ") order by airline,dpt_date"
report_list = []
conn = sqlite3.connect(DB)
with conn:
    df = pd.read_sql_query(query, conn)
df

query :  
        select scrap_date, airline, flt, dpt_date, dpt, arr, dpt_time, arr_time, fare, tax1, tax2, seat
        from airfare_scraped_data
        where scrap_date >= '2017070100'
        and dpt='CJU' and arr='GMP'
        and dpt_date >= '20170718' and dpt_date <= '20170724'
        


,airline,dpt_date,dpt,arr,flt,dpt_time,fare
0,7C,20170718,CJU,GMP,152,0845,25900
1,7C,20170718,CJU,GMP,104,0935,28900
2,7C,20170718,CJU,GMP,106,1045,31900
3,7C,20170718,CJU,GMP,108,1100,31900
4,7C,20170718,CJU,GMP,112,1145,31900
5,7C,20170718,CJU,GMP,114,1205,36900
6,7C,20170718,CJU,GMP,154,1220,36900
7,7C,20170718,CJU,GMP,116,1305,41900
8,7C,20170718,CJU,GMP,118,1355,46900
9,7C,20170718,CJU,GMP,120,1605,51900


In [164]:
## 경쟁사 가격 비교 하여 원하는 DataFrame 작성    
## 0. DataFrame 생성, fare의 '' 값 0 으로 보정
## DB 에서 읽어온 데이터를 이용하여 원하는 DataFrame 추출
columns = ['airline', 'dpt_date', 'dpt', 'arr', 'flt', 'dpt_time', 'fare']
df = df[columns]
## fare 컬럼에 빈 문자를 포함하는 경우 0 처리
if df['fare'].dtype == object:
    df.ix[df['fare'] == '','fare'] = 0 ## df[df == ''] = 0
## 1. 평균값 데이터 필터
dfmean = df.groupby(['airline','dpt_date','dpt','arr','flt','dpt_time']).agg(agg_mean)
dfmean = dfmean.reset_index()
## 2. ZE 기준 경쟁 플라이트 정보 처리
dfze = dfmean[dfmean['airline'] == 'ZE']
dfze = dfze.reset_index()[dfmean.columns]
## df에 dpt_mean(dpt_time을 분으로 계산한 필드) 추가 - 경쟁 플라이트 계산용
dfmean['dpt_mean'] = dfmin['dpt_time'].map(lambda x:int(x[:2])*60 + int(x[2:]))
## 3. 경쟁 플라이트 정보 찾기
airlines = set(dfmin['airline'].unique()) - set(['ZE'])
dpt_dates = dfze['dpt_date'].unique()
cal_min = lambda x:int(x[:2])*60+int(x[2:])
## 값을 채울 컬럼 생성
for airline in  airlines:
    dfze[airline+'-info'] = ''
    dfze[airline+'-fare'] = 0
print("checking other airline's competitive fare")
for dpt_date in dpt_dates:
    zeobj = dfze[dfze['dpt_date']==dpt_date]
    print('-- calculating on {} - data length : {}'.format(dpt_date,len(zeobj)),end=' = ')
    for airline in airlines:
        dfcmp = dfmean[(dfmean['airline'] == airline) & (dfmean['dpt_date'] == dpt_date)]
        if len(dfcmp) == 0:
            continue
        print(airline,end='/')
        for i in zeobj.index:
            dfcmp['dpt_cmp'] = abs(dfcmp['dpt_mean'] - cal_min(zeobj.ix[i,'dpt_time']))
            argmin = dfcmp['dpt_cmp'].argmin()
            dfze.ix[i,airline+'-info'] = dfcmp.ix[argmin,'airline'] +\
                    '-' + dfcmp.ix[argmin,'flt'] + '-' + dfcmp.ix[argmin,'dpt_time']
            dfze.ix[i,airline+'-fare'] = dfcmp.ix[argmin,'fare']
    print('')
dfze

checking other airline's competitive fare
-- calculating on 20170718 - data length : 20 = 7C/

d:\python\python35\lib\site-packages\ipykernel\__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


LJ/TW/
-- calculating on 20170719 - data length : 21 = 7C/LJ/TW/
-- calculating on 20170720 - data length : 20 = 7C/LJ/TW/
-- calculating on 20170721 - data length : 20 = 7C/LJ/TW/
-- calculating on 20170722 - data length : 19 = 7C/LJ/TW/
-- calculating on 20170723 - data length : 20 = 7C/LJ/TW/
-- calculating on 20170724 - data length : 21 = 7C/LJ/TW/


,airline,dpt_date,dpt,arr,flt,dpt_time,fare,7C-info,7C-fare,LJ-info,LJ-fare,TW-info,TW-fare
0,ZE,20170718,CJU,GMP,202,0850,0.000000,7C-112-1145,39840.000000,LJ-0310-0940,43575.000000,TW-714-1550,54640.000000
1,ZE,20170718,CJU,GMP,204,0810,33366.666667,7C-108-1100,41040.000000,LJ-0310-0940,43575.000000,TW-714-1550,54640.000000
2,ZE,20170718,CJU,GMP,206,0830,34366.666667,7C-112-1145,39840.000000,LJ-0310-0940,43575.000000,TW-714-1550,54640.000000
3,ZE,20170718,CJU,GMP,208,0930,41833.333333,7C-112-1145,39840.000000,LJ-0310-0945,41466.666667,TW-716-1705,59825.000000
4,ZE,20170718,CJU,GMP,210,1000,40300.000000,7C-154-1220,44075.000000,LJ-0310-0945,41466.666667,TW-716-1705,59825.000000
5,ZE,20170718,CJU,GMP,212,1235,49100.000000,7C-118-1355,51575.000000,LJ-0318-1310,49325.000000,TW-722-2025,54640.000000
6,ZE,20170718,CJU,GMP,214,1300,46433.333333,7C-118-1355,51575.000000,LJ-0318-1310,49325.000000,TW-722-2025,54640.000000
7,ZE,20170718,CJU,GMP,216,1330,45550.000000,7C-120-1605,58133.333333,LJ-0318-1310,49325.000000,TW-748-1955,57240.000000
8,ZE,20170718,CJU,GMP,218,1445,52420.000000,7C-122-1720,59075.000000,LJ-0320-1340,49240.000000,TW-746-1620,56575.000000
9,ZE,20170718,CJU,GMP,220,1455,51550.000000,7C-122-1720,59075.000000,LJ-0320-1340,49240.000000,TW-746-1620,56575.000000


In [165]:
dfze.head()

,airline,dpt_date,dpt,arr,flt,dpt_time,fare,7C-info,7C-fare,LJ-info,LJ-fare,TW-info,TW-fare
0,ZE,20170718,CJU,GMP,202,0850,0.000000,7C-112-1145,39840.0,LJ-0310-0940,43575.000000,TW-714-1550,54640.0
1,ZE,20170718,CJU,GMP,204,0810,33366.666667,7C-108-1100,41040.0,LJ-0310-0940,43575.000000,TW-714-1550,54640.0
2,ZE,20170718,CJU,GMP,206,0830,34366.666667,7C-112-1145,39840.0,LJ-0310-0940,43575.000000,TW-714-1550,54640.0
3,ZE,20170718,CJU,GMP,208,0930,41833.333333,7C-112-1145,39840.0,LJ-0310-0945,41466.666667,TW-716-1705,59825.0
4,ZE,20170718,CJU,GMP,210,1000,40300.000000,7C-154-1220,44075.0,LJ-0310-0945,41466.666667,TW-716-1705,59825.0


In [166]:
dfze.to_csv('cjugmp_20170728.csv')

In [154]:
dfze201 = dfze[dfze['flt'] == '201'][['dpt_date','flt','dpt_time','fare','7C-fare','LJ-fare','TW-fare']]
dfze201['key'] = dfze201['dpt_date']+'-'+dfze['flt']
dfze201

,dpt_date,flt,dpt_time,fare,7C-fare,LJ-fare,TW-fare,key
0,20170718,201,0615,0.0,44640.000000,41857.142857,44350.000000,20170718-201
20,20170719,201,0615,0.0,42016.666667,38516.666667,41516.666667,20170719-201
41,20170720,201,0615,39500.0,50325.000000,51840.000000,49240.000000,20170720-201
61,20170721,201,0615,60500.0,67875.000000,80833.333333,65442.857143,20170721-201
81,20170722,201,0615,66900.0,78200.000000,90833.333333,78157.142857,20170722-201
100,20170723,201,0615,75400.0,75871.428571,85833.333333,82866.666667,20170723-201
120,20170724,201,0615,76900.0,77260.000000,80833.333333,76014.285714,20170724-201


In [155]:
dfze201=dfze201.reset_index()[['key','dpt_date','flt','dpt_time','fare','7C-fare','LJ-fare','TW-fare']]
dfze201

,key,dpt_date,flt,dpt_time,fare,7C-fare,LJ-fare,TW-fare
0,20170718-201,20170718,201,0615,0.0,44640.000000,41857.142857,44350.000000
1,20170719-201,20170719,201,0615,0.0,42016.666667,38516.666667,41516.666667
2,20170720-201,20170720,201,0615,39500.0,50325.000000,51840.000000,49240.000000
3,20170721-201,20170721,201,0615,60500.0,67875.000000,80833.333333,65442.857143
4,20170722-201,20170722,201,0615,66900.0,78200.000000,90833.333333,78157.142857
5,20170723-201,20170723,201,0615,75400.0,75871.428571,85833.333333,82866.666667
6,20170724-201,20170724,201,0615,76900.0,77260.000000,80833.333333,76014.285714


In [156]:
dfze201= dfze201.set_index(['key','dpt_date','flt','dpt_time'])
dfze201

,,,,fare,7C-fare,LJ-fare,TW-fare
key,dpt_date,flt,dpt_time,,,,
20170718-201,20170718,201,0615,0.0,44640.000000,41857.142857,44350.000000
20170719-201,20170719,201,0615,0.0,42016.666667,38516.666667,41516.666667
20170720-201,20170720,201,0615,39500.0,50325.000000,51840.000000,49240.000000
20170721-201,20170721,201,0615,60500.0,67875.000000,80833.333333,65442.857143
20170722-201,20170722,201,0615,66900.0,78200.000000,90833.333333,78157.142857
20170723-201,20170723,201,0615,75400.0,75871.428571,85833.333333,82866.666667
20170724-201,20170724,201,0615,76900.0,77260.000000,80833.333333,76014.285714


In [158]:
dfze201.pivot_table(['dpt_date'],index=['key','flt','dpt_time'],columns=['fare','7C_fare','LJ_fare','TW_fare'],margins=True)

KeyError: "['dpt_date'] not in index"